# Datos

In [ ]:
#paquetes locales diseñados para nuestro caso de estudio
from localpacks.data import get_data 

#API maps
from geopy.geocoders import Nominatim
from shapely.geometry import Point, Polygon

#draw map streets
import osmnx as ox
import geopandas

#graficos
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-white')

### Inpeccionamos nuestros datos

In [ ]:
cordoba_sf = get_data() #obtenemos nuestros datos

In [ ]:
cordoba_sf.head(3)

Podemos observar que los datos corresponden a los de la descripción del **README.md**

In [ ]:
cordoba_sf.describe()

Vemos que no tenemos ninguna distribución normal en niguna de las features.

In [ ]:
cordoba_sf.info()

No tenemos datos nulos y posiblemente solo debamos cambiar el tipo de dato del `link` como integer.

In [ ]:
cordoba_sf.get_dtype_counts()

### Exploramos la API

In [ ]:
type(cordoba_sf.geometry) #tenemos un obj geopandas

In [ ]:
type(cordoba_sf.geometry[0]) #podemos observar que cada radio censal corresponde al área de un poligono

In [ ]:
cordoba_sf.geometry[0] #areas individuales

In [ ]:
cordoba_sf.geometry[0].area

In [ ]:
cordoba_sf.head(1)

### Creamos un punto en Cordoba

Seguimos explorando la API, y obtenemos un punto de Cordoba

In [ ]:
geolocator = Nominatim(user_agent="data_test") #creamos un nombre generico
location = geolocator.geocode("Santa Rosa 724, Alberdi, X5000 Córdoba, Argentina") #le pedimos la dirección de Cordoba

In [ ]:
print(' Dirección: {} \n longitud: {} \n latitud: {}'.format(location.address,location.longitude, location.latitude))

Vemos que devuelve las coordenadas de la Cda(Cordoba), Argentina.

In [ ]:
cordoba_point = Point(location.longitude, location.latitude)

### Creamos un mapa genérico de Cordoba

Utilizando un mapa vamos a verificar, donde quedó nuestro `cordoba_pint`

In [ ]:
G = ox.graph_from_place('Cordoba, Argentina', network_type='drive')
ox.save_graph_shapefile(G, 'openstreetmap_argentina', folder='.')

In [ ]:
cordoba_map = geopandas.read_file("openstreetmap_argentina/edges/")

In [ ]:
cordoba_map.head(3)

In [ ]:
cordoba_point_geo = geopandas.GeoSeries(cordoba_point)

In [ ]:
ax = cordoba_map.plot(linewidth=0.5, color='grey', figsize=(15, 10))
cordoba_point_geo.plot(ax=ax, markersize=200)
#ax.set(xlim=(2.25, 2.42), ylim=(48.815, 48.905));
#ax.set_axis_off()